# Evaluation Template

Run experiments/collect_evaluation_data.py to collect the data. Then run the following code to generate the plots.

In [ ]:
import os
import hydra

import torch
from experiments.collect_evaluation_data import dist_funcs
import matplotlib as mpl
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import seaborn as sns
from core.custom_dataset import CustomDataset

In [ ]:
from plotting import (
    fv_2d_grid_model_vs_parameters,
    fv_similarity_boxplots_by_dist_func,
    fv_2d_grid_step_vs_model,
    fv_mnist_output,
    fv_2d_grid_model_by_step_similarity, act_max_top_k_from_dataset,
)

In [ ]:
mpl.rcParams.update(mpl.rcParamsDefault)
plt.ioff()

np.random.seed(27)

os.environ["PATH"] += os.pathsep + "/Library/TeX/texbin"

In [ ]:
sns.set_theme()
sns.set_palette("pastel")
sns.set(font_scale=1.2)

In [ ]:
cfg_name = "config_mnist"

In [ ]:
load_path = f"../results/dataframes/{cfg_name}/"
save_path = f"../results/figures/{cfg_name}/"
os.makedirs(os.path.dirname(save_path), exist_ok=True)

In [ ]:
metadata = pd.read_pickle(f"{load_path}/metadata.pkl").iloc[0].to_dict()

## Manipulation

## Define Similarity Functions

### Qualitative Analysis: Plot 1

In [ ]:
results_df_by_step_basic = pd.read_pickle(f"{load_path}/results_df_by_step_basic.pkl")

In [ ]:
df = results_df_by_step_basic
df['model_dist'] = df['model']
for dist_str, dist_func, dist_str2 in dist_funcs[:-1]:
    dist_min = df[(df.iter == 0) & (df.step == df.step.max())][dist_str].min()
    bool_array = df[dist_str] == dist_min
    df[dist_str] = df[dist_str].astype(float)
    df[dist_str + "_corr"] = df[dist_str].copy().map('{:,.3f}'.format)
    df[dist_str + "_corr"][bool_array] = r'\textbf{' + str('{:,.3f}'.format(dist_min)) + r'}'
    df['model_dist'] = df['model_dist'] + "\n" + dist_str2 + ": " + df[dist_str + "_corr"]

In [ ]:
results_df_basic = results_df_by_step_basic[
    results_df_by_step_basic["step"] == results_df_by_step_basic["step"].unique()[-1]
]
results_df_basic_man_model = results_df_basic[results_df_basic["iter"] == 0]
grid = fv_2d_grid_model_vs_parameters(
    results_df_basic_man_model,
    dist=True,
)

plt.savefig(f"{save_path}/ssim_alpha_demo.png", bbox_inches="tight")
plt.show()

# Select Manipulation Model

In [ ]:
models = df.model.unique()
man_model = models[metadata["MAN_MODEL"]]
results_df_basic_man_model = results_df_basic[
    results_df_basic["model"] == man_model
]
results_df_basic_by_step_man_model = results_df_by_step_basic[
    results_df_by_step_basic["model"] == man_model
]
results_df_basic_original = results_df_basic[
    results_df_basic["model"] == "Original"
]
results_df_basic_by_step_original = results_df_by_step_basic[
    results_df_by_step_basic["model"] == "Original"
]

# Save Plot Images

In [ ]:
im = Image.fromarray(
    (results_df_basic_man_model.picture.values[0] * 255).squeeze().astype(np.uint8)
)
im.save(f"{save_path}/manipulated_fv.png")

In [ ]:
im = Image.fromarray(
    (results_df_basic_original.picture.values[0] * 255).squeeze().astype(np.uint8),
)
im.save(f"{save_path}/original_fv.png")

### Qualitative Analysis: Plot 2

In [ ]:
grid = fv_2d_grid_step_vs_model(
    results_df_basic_by_step_man_model,
    metadata["N_VIS"],
)
plt.savefig(f"{save_path}/man_am_progress.png")
plt.show()

### Qualitative Analysis: Plot 3

In [ ]:
results_10_neuron = pd.read_pickle(f"{load_path}/results_neuron_list.pkl")

In [ ]:
grid = fv_mnist_output(results_10_neuron)
plt.savefig(f"{save_path}/10_classes_before_after.png", bbox_inches="tight")
plt.show()

### Quantitative Analysis: Plot 4

In [ ]:
results_df_basic_100 = pd.read_pickle(f"{load_path}/results_df_basic_100.pkl")

In [ ]:
grid = fv_similarity_boxplots_by_dist_func(
    results_df_basic_100, 
    dist_funcs
)
grid.savefig(f"{save_path}/boxplot.png", bbox_inches="tight")
plt.show()

In [ ]:
eval_table = (
    results_df_basic_100.groupby(["model"])
    .describe(include=[float])
    .loc[:, (slice(None), ["mean", "std"])]
)

eval_table.columns = eval_table.columns.map("_".join)
for s in [d[0] for d in dist_funcs]:
    eval_table[s + "_mean"] = eval_table[s + "_mean"].map("${:,.3f}".format).astype(str)
    eval_table[s + "_std"] = eval_table[s + "_std"].map("{:,.3f}$".format).astype(str)
    eval_table[s] = eval_table[s + "_mean"] + "\pm" + eval_table[s + "_std"]

for s in ["acc"]:
    eval_table[s] = eval_table[s + "_mean"]

ssim_means = eval_table[dist_funcs[0][0] + "_mean"].str[1:].astype(float).copy().values

In [ ]:
eval_table = eval_table[
    ["acc"] + [d[0] for d in dist_funcs[::-1]]
    ]
eval_table["acc"] = eval_table["acc"].map("{:,.3f}".format).astype(str)
eval_table = eval_table.reset_index(drop=False)
eval_table["model"] = eval_table["model"].str[10:].str.replace("odel", "Original")
#eval_table["model"][eval_table["model"] != "Original"] = eval_table["model"][eval_table["model"] != "Original"].apply(sci_notation)

eval_table["model"][eval_table["model"] != "Original"] = eval_table["model"][
    eval_table["model"] != "Original"
    ].astype(str)
eval_table.columns = [
                         r"$\alpha$",
                         "Accuracy"] + [d[0] for d in dist_funcs[::-1]]


print(eval_table.to_latex(escape=False, index=False))

In [ ]:
results_df_basic = results_df_basic.reset_index(drop=True)
def plot_ssim_examples():
    fig, ax = plt.subplots()
    ax = imscatter(np.arange(1,len(models)), ssim_means[1:], results_df_basic['picture'][1:], zoom=0.95, ax=ax)
    ax.plot(np.arange(1,len(models)), ssim_means[1:])
    return ax, fig

def imscatter(x, y, images, ax=None, zoom=1):
    x, y = np.atleast_1d(x, y)
    artists = []
    i = 1
    for x0, y0 in zip(x, y):
        image = images[i].squeeze()
        if len(image.shape) == 2:
            cmap = 'gray'
        else:
            cmap = None
        im = OffsetImage(images[i].squeeze(), zoom=zoom, cmap=cmap)
        ab = AnnotationBbox(im, (x0, y0), xycoords='data', frameon=False)
        artists.append(ax.add_artist(ab))
        i += 1
    ax.update_datalim(np.column_stack([x, y]))
    ax.autoscale()
    return ax

ax, fig = plot_ssim_examples()
plt.rcParams.update({
        "text.usetex": True,
        "axes.titlesize": 10,
        "axes.labelsize": 13,
        "font.size": 10,
        "font.family": "Helvetica",
        "xtick.labelsize": 12,
        "ytick.labelsize": 10,
        'text.latex.preamble': r"\usepackage{amsmath}\usepackage{color}",
    })
ax.set_xticks(range(1, 10), eval_table[r"$\alpha$"][1:-1], rotation='vertical')
#plt.gca().set_aspect(7)
#plt.ylim([min(ssim_means)-0.08, max(ssim_means)+0.08])
#plt.xlim([0.4, 10.6])
ax.set_xlabel(r'Model', fontsize=21)
ax.set_ylabel(r'SSIM', fontsize=12)
plt.savefig(f"{save_path}/ssim_dynamics.png", bbox_inches="tight")
plt.show()
plt.clf()

### Quantitative Analysis: Plot 5

In [ ]:
results_df_by_step_basic_100 = pd.read_pickle(f"{load_path}/results_df_by_step_basic_100.pkl")

In [ ]:
grid = fv_2d_grid_model_by_step_similarity(
    results_df_by_step_basic_100,
    dist_funcs,
)
grid.savefig(f"{save_path}/similarity_step.png", bbox_inches="tight")
plt.show()

### Natural Images

In [ ]:
cfg = results_df_basic_original.iloc[0]["cfg"]
denormalize = hydra.utils.instantiate(cfg.data.denormalize)

class_dict_file = cfg.data.get("class_dict_file", None)
if class_dict_file is not None:
    class_dict_file = "." + class_dict_file

train_dataset, test_dataset = hydra.utils.instantiate(
    cfg.data.load_function, path=cfg.data_dir + cfg.data.data_path
)
test_loader = torch.utils.data.DataLoader(
    CustomDataset(test_dataset, class_dict_file),
    batch_size=cfg.batch_size,
    shuffle=False,
    num_workers=0,
)

In [ ]:
print("Before")
fig1 = act_max_top_k_from_dataset(
    results_df_basic_original.iloc[0]["top_k_names"],
    denormalize,
    test_loader.dataset,
)
fig1.savefig(f"{save_path}/top_4_before.png", bbox_inches="tight")
plt.show()

In [ ]:
print("After")
fig1 = act_max_top_k_from_dataset(
    results_df_basic_man_model.iloc[0]["top_k_names"],
    denormalize,
    test_loader.dataset,
)
fig1.savefig(f"{save_path}/top_4_before.png", bbox_inches="tight")
plt.show()

# Jaccard similarity coefficient

In [ ]:
print("After: ", results_df_basic["jaccard"])

# AUC

In [ ]:
print("Before: ", results_df_basic_original.iloc[0]["auc"])
print("After: ", results_df_basic_man_model.iloc[0]["auc"])